In [1]:
# Load the Drive helper and mount
from google.colab import drive

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')
data_path = '/content/drive/My Drive/Datathon/'

Mounted at /content/drive


In [8]:
!pip install pytorch_lightning

     |████████████████████████████████| 582 kB 4.5 MB/s 
     |████████████████████████████████| 136 kB 29.6 MB/s 
     |████████████████████████████████| 398 kB 24.3 MB/s 
     |████████████████████████████████| 596 kB 28.6 MB/s 
     |████████████████████████████████| 1.1 MB 18.3 MB/s 
     |████████████████████████████████| 271 kB 46.2 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 144 kB 44.9 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow

In [9]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.4 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
     |████████████████████████████████| 895 kB 41.7 MB/s 
     |████████████████████████████████| 6.5 MB 32.5 MB/s 


In [10]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import torch
import torchmetrics
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast as BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix, roc_auc_score
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline
%config InlineBackend.figure_format='retina'
RANDOM_SEED = 42
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
pl.seed_everything(RANDOM_SEED)

Global seed set to 42


42

In [11]:
import sys
from collections import Counter
from nltk.tokenize import word_tokenize
import nltk
from nltk.tokenize.casual import TweetTokenizer
from nltk.corpus import stopwords, wordnet
import string
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.probability import FreqDist
from wordcloud import WordCloud, ImageColorGenerator
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
import pickle
from nltk.stem import SnowballStemmer
import spacy

In [12]:
df = pd.read_csv('/content/drive/My Drive/Datathon2022/data/' + 'labels_racism.csv', sep='|'  , engine='python')

In [13]:
df_train = df[df['label'] != 'unknown'][['message', 'label']]

In [15]:
# converting to binary data
dummies = pd.get_dummies(df_train['label'])
print(dummies)

      non-racist  racist
0              1       0
1              1       0
2              1       0
3              1       0
4              1       0
...          ...     ...
9286           0       1
9287           0       1
9288           0       1
9289           0       1
9290           1       0

[8870 rows x 2 columns]


In [16]:
# display result
df_two = pd.concat((df_train, dummies), axis=1)
df_two = df_two.drop(["label"], axis=1)
df_two = df_two.drop(["non-racist"], axis=1)
df_train = df_two.rename(columns={"racist": "label"})

In [18]:
df_test = pd.read_csv(('/content/drive/My Drive/Datathon2022/data/evaluation_sample.csv'), sep='|' , engine='python')
df_test.head()

,message,label
0,Feméride' (DCV): hoy se cumplen 60 años del es...,non-racist
1,Pegan una paliza a dos policías en Zaragoza Pe...,racist
2,@A_ausdenRuthen @SimonLevyMx Pinches musulmane...,non-racist
3,@Puri95702837 @yeilouondeflo @PLAyer__slaYER @...,non-racist
4,@HamzaMaca a quien juzgo? a ti? has empezado a...,non-racist


In [19]:
# converting to binary data
dummies_test = pd.get_dummies(df_test['label'])
print(dummies_test)

    non-racist  racist
0            1       0
1            0       1
2            1       0
3            1       0
4            1       0
5            0       1
6            1       0
7            1       0
8            0       1
9            0       1
10           0       1
11           1       0
12           1       0
13           1       0
14           1       0
15           1       0
16           0       1
17           1       0
18           1       0
19           1       0
20           0       1
21           1       0
22           0       1
23           1       0
24           0       1
25           1       0
26           1       0
27           1       0
28           0       1
29           0       1
30           0       1
31           0       1
32           1       0
33           1       0
34           1       0
35           0       1
36           0       1
37           0       1
38           0       1
39           1       0
40           1       0
41           1       0
42         

In [20]:
# display result
df_two = pd.concat((df_test, dummies_test), axis=1)
df_two = df_two.drop(["label"], axis=1)
df_two = df_two.drop(["non-racist"], axis=1)
df_test = df_two.rename(columns={"racist": "label"})

In [21]:
df_train.shape, df_test.shape

((8870, 2), (59, 2))

In [22]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("skimai/spanberta-base-cased")

Downloading:   0%|          | 0.00/932k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/500k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523 [00:00<?, ?B/s]

In [23]:
MAX_TOKEN_COUNT = 300

In [24]:
class RacistDataset(Dataset):
    def __init__(self, data: pd.DataFrame, tokenizer: BertTokenizer, max_token_len: int = 128):
            self.tokenizer = tokenizer
            self.data = data
            self.max_token_len = max_token_len
            
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]
        comment_text = data_row.message + data_row.stemm_str
        label = data_row.label
        encoding = self.tokenizer.encode_plus(
          comment_text,
          add_special_tokens=True,
          max_length=self.max_token_len,
          return_token_type_ids=False,
          padding="max_length",
          truncation=True,
          return_attention_mask=True,
          return_tensors='pt',
        )
        return dict(
          comment_text=comment_text,
          input_ids=encoding["input_ids"].flatten(),
          attention_mask=encoding["attention_mask"].flatten(),
          label=torch.tensor(label)
        )

In [26]:
train_dataset = RacistDataset(
  df_train,
  tokenizer,
  max_token_len=MAX_TOKEN_COUNT
)

In [27]:
from transformers import RobertaConfig, RobertaModel

bert_model = RobertaModel.from_pretrained("skimai/spanberta-base-cased")

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

In [28]:
from torchmetrics import AUROC
auroc = AUROC(pos_label=1)

/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [29]:
class RacistDataModule(pl.LightningDataModule):
    def __init__(self, train_df, test_df, tokenizer, batch_size=8, max_token_len=128):
        super().__init__()
        self.batch_size = batch_size
        self.df_train = df_train
        self.df_test = df_test
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len
        
    def setup(self, stage=None):
        self.train_dataset = RacistDataset(
          self.df_train,
          self.tokenizer,
          self.max_token_len
        )
        self.test_dataset = RacistDataset(
          self.df_test,
          self.tokenizer,
          self.max_token_len
        )
        
    def train_dataloader(self):
        return DataLoader(
          self.train_dataset,
          batch_size=self.batch_size,
          shuffle=True
        )
    
    def test_dataloader(self):
        return DataLoader(
          self.test_dataset,
          batch_size=self.batch_size
        )

In [30]:
N_EPOCHS = 15
BATCH_SIZE = 12
data_module = RacistDataModule(
  df_train,
  df_test,
  tokenizer,
  batch_size=BATCH_SIZE,
  max_token_len=MAX_TOKEN_COUNT
)

In [31]:
class RacistTagger(pl.LightningModule):
    def __init__(self, n_classes: int, n_training_steps=None, n_warmup_steps=None):
        super().__init__()
        self.bert = RobertaModel.from_pretrained("skimai/spanberta-base-cased")
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.criterion = nn.BCELoss()
        
    def forward(self, input_ids, attention_mask, labels=None):
        output = self.bert(input_ids, attention_mask=attention_mask)
        output = self.classifier(output.pooler_output)
        output = torch.sigmoid(output)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels.unsqueeze(1).float())
        return loss, output
    
    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        label = batch["label"]
        loss, outputs = self(input_ids, attention_mask, label)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return {"loss": loss, "predictions": outputs, "labels": label}
    

    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        label = batch["label"]
        loss, outputs = self(input_ids, attention_mask, label)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss
    
    def training_epoch_end(self, outputs):
        labels = []
        predictions = []
        for output in outputs:
            for out_labels in output["labels"].detach().cpu():
                    labels.append(out_labels)
            for out_predictions in output["predictions"].detach().cpu():
                    predictions.append(out_predictions)
        labels = torch.stack(labels).int()
        #print('LABELS', labels)
        predictions = torch.round(torch.stack(predictions))
        #print('PREDICTIONS', predictions)
        class_roc_auc = roc_auc_score(predictions, labels)
        self.logger.experiment.add_scalar(f"_roc_auc/Train", class_roc_auc, self.current_epoch)
            
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=2e-5)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=self.n_warmup_steps, num_training_steps=self.n_training_steps)
        return dict(optimizer=optimizer, lr_scheduler=dict(scheduler=scheduler,interval='step'))

In [37]:
model_loaded = pickle.load(open(data_path + "roberta.pkl", "rb"))

In [ ]:
steps_per_epoch=len(df_train) // BATCH_SIZE
total_training_steps = steps_per_epoch * N_EPOCHS

warmup_steps = total_training_steps // 5
warmup_steps, total_training_steps

(1371, 6855)

In [ ]:
model = RacistTagger(
  n_classes=1,
  n_warmup_steps=warmup_steps,
  n_training_steps=total_training_steps
)

In [ ]:
checkpoint_callback = ModelCheckpoint(
  dirpath="checkpoints",
  filename="best-checkpoint",
  save_top_k=1,
  verbose=True,
  monitor="train_loss",
  mode="min"
)

In [ ]:
logger = TensorBoardLogger("lightning_logs", name="racist-comments")

In [ ]:
early_stopping_callback = EarlyStopping(monitor='train_loss', patience=2)

In [ ]:
trainer = pl.Trainer(
  logger=logger,
  checkpoint_callback=checkpoint_callback,
  callbacks=[early_stopping_callback],
  max_epochs=N_EPOCHS,
  gpus=1,
  progress_bar_refresh_rate=30
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f575d089210>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f575d089210>)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=30)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Tr

In [ ]:
trainer.fit(model, data_module)

Missing logger folder: lightning_logs/racist-comments
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name       | Type         | Params
--------------------------------------------
0 | bert       | RobertaModel | 124 M 
1 | classifier | Linear       | 769   
2 | criterion  | BCELoss      | 0     
--------------------------------------------
124 M     Trainable params
0         Non-trainable params
124 M     Total params
498.586   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

In [ ]:
trained_model = RacistTagger.load_from_checkpoint(
  trainer.checkpoint_callback.best_model_path,
  n_classes=1
)

In [ ]:
trained_model.eval()
trained_model.freeze()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
trained_model = trained_model.to(device)
val_dataset = RacistDataset(
  df_test,
  tokenizer,
  max_token_len=MAX_TOKEN_COUNT
)
predictions = []
labels = []
for item in tqdm(val_dataset):
  _, prediction = trained_model(
    item["input_ids"].unsqueeze(dim=0).to(device),
    item["attention_mask"].unsqueeze(dim=0).to(device)
  )
  predictions.append(prediction.flatten())
  labels.append(item["label"].int())
predictions = torch.stack(predictions).detach().cpu()
labels = torch.stack(labels).detach().cpu()

  0%|          | 0/59 [00:00<?, ?it/s]

In [ ]:
df_test['label'] = torch.round(predictions)
df_test['probability'] = predictions
df_test.head()

,message,stemm_str,label,probability
0,""" la Policía Nacional busca a dos jóvenes magr...",polic nacional busc dos joven magreb presunt a...,1.0,0.999788
1,.@joanribo ya ha implantado con éxito el model...,@joanrib implant exit model colau casc histor ...,1.0,0.999680
2,@A_ausdenRuthen @SimonLevyMx Pinches musulmane...,@a_ausdenruth @simonlevymx pinch musulman igual 😂,0.0,0.000760
3,@BatShem Hace mucho que no se oye algo tan ant...,@batshem hac oye tan antisemit tip lament judi...,0.0,0.000256
4,@CHVNoticias Espero le muestre el dedo a todos...,@chvnotici esper muestr ded inmigr ilegal terr...,1.0,0.999730
